In [39]:
# Import all required libraries

import nltk
import numpy as np
import re
import pandas as pd 
import pylab as pl
import matplotlib.pyplot as plt

from nltk.tokenize import WordPunctTokenizer
#from bs4 import BeautifulSoup
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD

from sklearn import metrics
from mpl_toolkits.mplot3d import Axes3D

from matplotlib import pyplot

#plt.style.use('fivethirtyeight')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

!pip install datashader
import datashader as ds
import datashader.transfer_functions as tf


In [40]:
from tweepy import API
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream


import tweepy


import twitter_credentials
import numpy as np
import pandas as pd


class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth


class TwitterStreamer():

    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()

    def stream_tweets(self, hash_tag_list):

        listener = TwitterListener()
        auth = self.twitter_autenticator.authenticate_twitter_app()
        stream = Stream(auth, listener)


        stream.filter(track=hash_tag_list)

class TweetAnalyzer():

    def tweets_to_data_frame(self, tweets):
        #for tweet in tweets:
            #print(tweet.full_text)

        df = pd.DataFrame(data=[tweet.full_text for tweet in tweets], columns=['Tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.full_text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])

        return df

if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = tweepy.Cursor(api.search, q=['Covishield','Covaxin'], lang="en",
                           tweet_mode='extended').items(200)
    list_tweets = [tweet for tweet in tweets]

    #print(dir(tweets[0]))
    #print(tweets[0].retweet_count)

    df = tweet_analyzer.tweets_to_data_frame(list_tweets)

    print(df.head(10))
    #print(df['Tweets'])


                                              Tweets                   id  \
0  @digvijaay46 @JoatAyushi @ANI India is plannin...  1379489101976494081   
1  Very nicely explained if you really want to kn...  1379488295495688192   
2  RT @KiranKS: India is planning to double per m...  1379487728807501825   
3  India currently produces #Covishield (a local ...  1379487049863036931   
4  RT @KiranKS: India is planning to double per m...  1379485612151992321   
5  RT @gauripokhariyal: When Covid19 vaccination ...  1379485118444630022   
6  @virsanghvi Mr Sanghvi has lot on his plate fr...  1379484897149034498   
7  Oxford-AstraZeneca COVID-19 vaccine: 'Review o...  1379483898371993600   
8  @hisaurabhshah Large portion, a very large por...  1379482749774438400   
9  RT @KiranKS: India is planning to double per m...  1379482584745304074   

   len                date               source  likes  retweets  
0  287 2021-04-06 17:40:06  Twitter for Android      0         0  
1   95 2021-04-06 

In [41]:
df.head(2)

,Tweets,id,len,date,source,likes,retweets
0,@digvijaay46 @JoatAyushi @ANI India is plannin...,1379489101976494081,287,2021-04-06 17:40:06,Twitter for Android,0,0
1,Very nicely explained if you really want to kn...,1379488295495688192,95,2021-04-06 17:36:53,Twitter for Android,3,0


In [42]:
# Number of users with multiple tweets

df["id"].value_counts(ascending=False).head(2)

1379464893187825667    1
1379476847486935040    1
Name: id, dtype: int64

In [43]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt 

In [44]:
df['Clean_text'] = np.vectorize(remove_pattern)(df['Tweets'], "@[\w]*")

In [45]:
# remove special characters, numbers, punctuations
df['Clean_text'] = df['Clean_text'].str.replace("[^a-zA-Z#]", " ")

<ipython-input-45-46be3fd0461e>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Clean_text'] = df['Clean_text'].str.replace("[^a-zA-Z#]", " ")


In [46]:
df['Clean_text'] = df['Clean_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [47]:
tokenized_tweet = df['Clean_text'].apply(lambda x: x.split())

In [48]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet.head()

0    [india, plan, doubl, month, vaccin, product, r...
1    [veri, nice, explain, realli, want, know, cova...
2    [india, plan, doubl, month, vaccin, product, r...
3    [india, current, produc, #covishield, local, v...
4    [india, plan, doubl, month, vaccin, product, r...
Name: Clean_text, dtype: object

In [49]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

df['Clean_text'] = tokenized_tweet

In [50]:
df.loc[:,('Tweets','Clean_text')]

,Tweets,Clean_text
0,@digvijaay46 @JoatAyushi @ANI India is plannin...,india plan doubl month vaccin product right pu...
1,Very nicely explained if you really want to kn...,veri nice explain realli want know covaxin cov...
2,RT @KiranKS: India is planning to double per m...,india plan doubl month vaccin product right pu...
3,India currently produces #Covishield (a local ...,india current produc #covishield local version...
4,RT @KiranKS: India is planning to double per m...,india plan doubl month vaccin product right pu...
...,...,...
195,@nightin32627875 @Liliana25099923 @StrohJulian...,correct mostli covishield there shortag covaxin
196,@Vickypats66 @SidShirole @sudhirmehtapune @apa...,actual pune covaxin avail think everywher covi...
197,Covaxin is not available anywhere in Rajnandga...,covaxin avail anywher rajnandgaon chhattisgarh...
198,RT @Raghu610: Covishield efficacy 70% 👍👍\nCova...,covishield efficaci covaxin efficaci


In [51]:
df[df.Tweets.isnull()]


,Tweets,id,len,date,source,likes,retweets,Clean_text


In [52]:
df[df.Clean_text.isnull()]

,Tweets,id,len,date,source,likes,retweets,Clean_text


In [53]:
# Number of unique tweets
unique_clean_text = df.Clean_text.unique()
unique_full_text = df.Tweets.unique()
print(len(unique_clean_text))
print(len(unique_full_text))
print(len(df))

128
140
200


In [54]:
df.drop_duplicates(subset=['Clean_text'], keep = 'first',inplace= True)

In [55]:
df.reset_index(drop=True,inplace=True)

In [56]:
df['Clean_text_length'] = df['Clean_text'].apply(len)
df.head()

,Tweets,id,len,date,source,likes,retweets,Clean_text,Clean_text_length
0,@digvijaay46 @JoatAyushi @ANI India is plannin...,1379489101976494081,287,2021-04-06 17:40:06,Twitter for Android,0,0,india plan doubl month vaccin product right pu...,162
1,Very nicely explained if you really want to kn...,1379488295495688192,95,2021-04-06 17:36:53,Twitter for Android,3,0,veri nice explain realli want know covaxin cov...,69
2,RT @KiranKS: India is planning to double per m...,1379487728807501825,140,2021-04-06 17:34:38,Twitter for iPhone,0,32,india plan doubl month vaccin product right pu...,79
3,India currently produces #Covishield (a local ...,1379487049863036931,220,2021-04-06 17:31:56,Twitter Web App,0,0,india current produc #covishield local version...,146
4,RT @gauripokhariyal: When Covid19 vaccination ...,1379485118444630022,139,2021-04-06 17:24:16,Twitter for Android,0,6,when covid vaccin drive start want inject with...,70


In [57]:
from textblob import TextBlob

In [58]:
TextBlob('Still enjoying the almost nice day out.  Now its down to +13, and calling for rain...').sentiment

Sentiment(polarity=0.3148148148148148, subjectivity=0.6296296296296297)

In [59]:
c=TextBlob("This was because he wasted time listing the failures of Congress prime  ministers. This was unnecessary because they are history and you, Narendra Modi, are the first Prime Minister in 30 years to be given a full mandate by the people of India")

In [60]:
def calculate_sentiment(Clean_text):
    return TextBlob(Clean_text).sentiment

In [61]:
def calculate_sentiment_analyser(Clean_text):    
    return analyser.polarity_scores(Clean_text)

In [62]:
df['sentiment']=df.Clean_text.apply(calculate_sentiment)
df['sentiment_analyser']=df.Clean_text.apply(calculate_sentiment_analyser)


s = pd.DataFrame(index = range(0,len(df)),columns= ['compound_score','compound_score_sentiment'])

for i in range(0,len(df)): 
  s['compound_score'][i] = df['sentiment_analyser'][i]['compound']
  
  if (df['sentiment_analyser'][i]['compound'] <= -0.05):
    s['compound_score_sentiment'][i] = 'Negative'    
  if (df['sentiment_analyser'][i]['compound'] >= 0.05):
    s['compound_score_sentiment'][i] = 'Positive'
  if ((df['sentiment_analyser'][i]['compound'] >= -0.05) & (df['sentiment_analyser'][i]['compound'] <= 0.05)):
    s['compound_score_sentiment'][i] = 'Neutral'
    
df['compound_score'] = s['compound_score']
df['compound_score_sentiment'] = s['compound_score_sentiment']
df.head(4)


,Tweets,id,len,date,source,likes,retweets,Clean_text,Clean_text_length,sentiment,sentiment_analyser,compound_score,compound_score_sentiment
0,@digvijaay46 @JoatAyushi @ANI India is plannin...,1379489101976494081,287,2021-04-06 17:40:06,Twitter for Android,0,0,india plan doubl month vaccin product right pu...,162,"(0.2857142857142857, 0.5357142857142857)","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,Neutral
1,Very nicely explained if you really want to kn...,1379488295495688192,95,2021-04-06 17:36:53,Twitter for Android,3,0,veri nice explain realli want know covaxin cov...,69,"(0.6, 1.0)","{'neg': 0.0, 'neu': 0.661, 'pos': 0.339, 'comp...",0.4767,Positive
2,RT @KiranKS: India is planning to double per m...,1379487728807501825,140,2021-04-06 17:34:38,Twitter for iPhone,0,32,india plan doubl month vaccin product right pu...,79,"(0.2857142857142857, 0.5357142857142857)","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,Neutral
3,India currently produces #Covishield (a local ...,1379487049863036931,220,2021-04-06 17:31:56,Twitter Web App,0,0,india current produc #covishield local version...,146,"(0.0, 0.2)","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,Neutral


In [63]:
df.compound_score_sentiment.value_counts()

Neutral     65
Positive    44
Negative    19
Name: compound_score_sentiment, dtype: int64

In [64]:
df['Clean_text'].head()

0    india plan doubl month vaccin product right pu...
1    veri nice explain realli want know covaxin cov...
2    india plan doubl month vaccin product right pu...
3    india current produc #covishield local version...
4    when covid vaccin drive start want inject with...
Name: Clean_text, dtype: object

In [65]:
#tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# Considering 3 grams and mimnimum frq as 0
tf_idf_vect = CountVectorizer(analyzer='word',ngram_range=(1,1),stop_words='english', min_df = 0.0001)
tf_idf_vect.fit(df['Clean_text'])
desc_matrix = tf_idf_vect.transform(df["Clean_text"])


In [66]:
# implement kmeans

from sklearn.cluster import KMeans
num_clusters = 3
km = KMeans(n_clusters=num_clusters)
km.fit(desc_matrix)
clusters = km.labels_.tolist()

In [67]:
# create DataFrame films from all of the input files.
tweets = {'Tweet': df["Clean_text"].tolist(), 'Cluster': clusters}
frame = pd.DataFrame(tweets, index = [clusters])
frame

,Tweet,Cluster
1,india plan doubl month vaccin product right pu...,1
0,veri nice explain realli want know covaxin cov...,0
1,india plan doubl month vaccin product right pu...,1
0,india current produc #covishield local version...,0
2,when covid vaccin drive start want inject with...,2
...,...,...
2,indian medic associ write prime minist narendr...,2
0,they take dolo time base symptom note covishie...,0
0,correct mostli covishield there shortag covaxin,0
0,actual pune covaxin avail think everywher covi...,0


In [68]:
frame['Cluster'].value_counts()

0    67
2    55
1     6
Name: Cluster, dtype: int64

In [69]:
frame[frame['Cluster'] == 1]

,Tweet,Cluster
1,india plan doubl month vaccin product right pu...,1
1,india plan doubl month vaccin product right pu...,1
1,avail reliabl inform current product capac cov...,1
1,india plan doubl month vaccin product right pu...,1
1,serum institut make million dose covishield ev...,1
1,ndtv channel report india vaccin last current ...,1


In [70]:
frame[frame['Cluster'] == 2]

,Tweet,Cluster
2,when covid vaccin drive start want inject with...,2
2,oxford astrazeneca covid vaccin review conclus...,2
2,http xscmxp while those have been administ cov...,2
2,with vaccin centr jab hour shift crore vaccin ...,2
2,which vaccin better covishield covaxin,2
2,colleagu corona after both dose hospitalis som...,2
2,ishkaranji covishield vaccin own view astra ze...,2
2,india total dose contract suppli contract covi...,2
2,hack figur where covishield covaxin near login...,2
2,time give higher prefer #covaxin have higher e...,2
